# 토픽모델링을 통한 키워드 분석

## 1. 사용할 라이브러리/데이터 불러오기 

In [ ]:
!pip install gensim==3.8.3

In [ ]:
import sys
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm  # progress bar
from konlpy.tag import Mecab; mecab = Mecab()

import numpy as np
import pandas as pd
import string
import re
import warnings
from gensim import corpora
from gensim import models
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
%matplotlib inline

## 운영체제별 글꼴 세팅

# 그래프를 이쁘게 그리기 위한 코드입니다. 한글 글꼴을 추가합니다.

%matplotlib inline  

import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
import seaborn as sns
mpl.rcParams['axes.unicode_minus'] = False

sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져옵니다.
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

In [ ]:
# Colab Notebooks
base_path = "/content/drive/MyDrive/Colab Notebooks/data/topic_modeling/"

# 아까 크롤링한 파일
## 크롤링한 파일이 바뀌면, 경로도 수정되어야 합니다.
data_path = 

In [ ]:
import pandas as pd

documents = 
documents

## 2. 전처리 함수 정의하기 

In [ ]:
def text_cleaning(doc):
    # 한국어를 제외한 글자를 제거하는 패턴.
    #doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    
    # 특수문자를 제거하는 패턴.
    doc = re.sub("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]", " ", doc)
    
    # 영문 빼고 모두 제거하는 패턴.
    #doc = doc.replace("\n", " ")
    #doc = re.sub("[^A-Za-z ]", "", doc)
    
    return doc

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # SW.add("있다")
    SW.add("있어요")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path, encoding="utf-8") as f:
        for word in f:
            SW.add(word.strip())
            
    return SW

def text_tokenizing(doc, tokenizer):
    """
    Input Parameter :
    
    doc - tokenizing 하는 실제 데이터.
    tokenizer - token의 단위.
    """
    # 불용어(SW)에 포함되어 있지 않고, 두 글자 이상인 token(형태소)만 사용하겠다.
    
    if tokenizer == "words":
        return [token for token in doc.split() if (token not in SW) and (len(token) >= 2)]
    
    elif tokenizer == "nouns":
        return [token for token in mecab.nouns(doc) if (token not in SW) and (len(token) >= 2)] # 주어진 텍스트에서 명사만 뽑아서 리스트로 반환.
        
    elif tokenizer == "morphs":
        return [token for token in mecab.morphs(doc) if (token not in SW) and (len(token) >= 2)] # 주어진 텍스트에서 형태소 단위로 잘라서 리스트로 반환.

In [ ]:
tokenizer = 
SW = define_stopwords(base_path + "stopwords-ko.txt")
tokenized_documents = [text_tokenizing(text_cleaning(doc), tokenizer=tokenizer) for doc in documents]

In [ ]:
print(len(tokenized_documents))
print(tokenized_documents[0])

### gensim LDA model을 사용하기 위한 자료구조 생성. 

In [ ]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_documents)
# 문서-단어 행렬 생성
corpus = [dictionary.doc2bow(document) for document in tokenized_documents]

In [ ]:
NUM_TOTAL_WORDS = len(dictionary)
print(dictionary)
print(NUM_TOTAL_WORDS)

## 3. 토픽 모델링(LDA Model)


- LDA(Latent Dirichlet Allocation) : 단어들의 조합(토픽)이 하나의 토픽을 구성하고, 각 단어가 그 토픽에 할당될 확률을 계산합니다.

- LDA는 사람이 글을 쓰는 과정을 다음과 같이 가정합니다.

1) 여러 개의 토픽을 정한다.

2) 토픽 하나를 고릅니다.

3) 토픽 내에 속하는 단어 하나를 고릅니다.

4) 해당 단어를 글에 적습니다.

5) 2번 과정부터 반복하면서 글을 적습니다.


-> LDA는 사람이 글을 쓰는 과정을 따라하면서 글을 생성하는 과정을 학습합니다. (Generative Model)

-> LDA는 생성을 통해서, 저자가 생각한 토픽의 분포(단어들의 확률 분포)를 찾습니다.

In [ ]:
# ldamodel 선언


# Coherence model 선언 (LDA의 정량 평가 지표.)


In [ ]:
# 토픽 개수와 토픽 별 상위 추출 단어 개수 지정.

def build_doc_term_mat(documents):
    """주어진 문서 집합으로 문서-어휘 행렬을 만들어 돌려준다."""
    
    print_log_msg("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]

    return corpus, dictionary


def print_topic_words(model):
    """토픽별 토픽 단어들을 화면에 인쇄한다."""
    
    print_log_msg("Printing topic words.")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, 30)
        print("Topic ID: {}".format(topic_id))

        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))

        print("\n")


def print_log_msg(msg):
    """로그 메시지를 출력한다."""
    
    print(msg, flush=True)

In [ ]:
def compute_coherence(dictionary, corpus, texts, start, end, step):
    '''
    start, end+1, step에 해당하는 토픽 개수를 입력받아서 LdaModel을 수행하고, 그 때의 Coherence Score를 함께 돌려주는 함수.
    '''
    coherence_score_list = []
    model_list = []
    for num_topics in tqdm(range(start, end+1, step)):
        model = 
        
        model_list.append(model)
        coherence_model = CoherenceModel(model=model, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
        coherence_score_list.append(coherence_model.get_coherence())
        
    return model_list, coherence_score_list

In [ ]:
# LDA에서 가장 중요한 파라미터 = K(토픽 개수)
start, end, step = [int(x) for x in input("원하는 토픽 갯수들을 입력하세요(e.g. 2,5,1)").split(",")]
start, end, step

In [ ]:
corpus, dictionary = build_doc_term_mat(tokenized_documents)
print(len(corpus), len(dictionary))


model_list, coherence_scores = compute_coherence(dictionary=dictionary, corpus=corpus, 
                                                texts=tokenized_documents, start=start, end=end, step=step)

In [ ]:
coherence_list = coherence_scores
label = "Coherenece Score(C_V)"

x = range(start, end+1, step)
plt.figure(figsize=(16, 12))
plt.xticks(x)
plt.plot(x, coherence_list, label=label)
plt.scatter(x, coherence_list)
plt.title(f"Coherence Scores for LDA with {file_name}")
plt.xlabel("Num Topics")
plt.ylabel("Coherence Score")
plt.legend(loc='best')
plt.show()

In [ ]:
selected_model = model_list[np.argmax(coherence_list)] # coherence score가 가장 높은 LDA model을 selected_model로 할당.
selected_model.num_topics

### 4. pyLDAvis를 이용한 시각화 

In [ ]:
!pip install pyldavis==3.2.1

In [ ]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare()
data